In [1]:
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain
import json
import re
from tqdm import tqdm
from pyspark.sql.functions import broadcast
from datetime import datetime, timedelta
import os
os.chdir('/home/y0c07y1/rpc_feature_extraction/rpc_model/')
from utils.utils import *
import gcsfs

In [2]:
import pyspark
from pyspark.sql.functions import lit
from pyspark.sql import functions as func
import pyspark.sql as pss
from pyspark.sql.functions import col
from pyspark.sql import types as t
from pyspark.sql.types import (
    StringType,
    MapType,
    LongType,
    DoubleType,
    FloatType,
    StructField,
    StructType,
    IntegerType,
    ArrayType
)
#spark = pss.SparkSession.builder.appName("dctr_test3").config(conf=pyspark.SparkConf().setAll([('spark.yarn.queue', 'root.critical')])).enableHiveSupport().getOrCreate()
def gen_spark_session(ds):
    ds_nodash = ds.replace("-", "")

    return pyspark.sql.SparkSession.builder.config(
            "hive.exec.dynamic.partition", True,
        ).config(
            "hive.exec.dynamic.partition.mode", "nonstrict"
        ).config('mapreduce.input.fileinputformat.input.dir.recursive',True
        ).config('spark.hive.mapred.supports.subdirectories',True
        ).config("spark.sql.crossJoin.enabled", "true"
        ).config("spark.sql.broadcastTimeout", "36000"
        ).config("spark.sql.sources.partitionOverwriteMode","dynamic"
        ).config("spark.hadoop.orc.overwrite.output.file",True
        ).config("spark.yarn.executor.memoryOverhead", '20g'
        ).config("spark.executor.memory",'12g'
        ).config("spark.driver.memory",'12g'
        ).config("spark.dynamicAllocation.enabled",'true'
        ).config("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation",'true'
        ).config("spark.sql.hive.convertMetastoreOrc", 'false'
        ).appName(APP_NAME % ds_nodash).enableHiveSupport().getOrCreate()


APP_NAME = 'rpc_feature_extractionjupyter_%s'
spark = gen_spark_session('2022-12/7')
spark.sparkContext.setLogLevel("ERROR")

# spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 10000000)

In [3]:
from __future__ import absolute_import

from pyspark.sql import functions as func
from pyspark import SparkConf, SparkContext
from pyspark.sql import HiveContext, SparkSession
from pyspark.sql.functions import col, struct, to_json, udf, expr
from pyspark.sql.window import Window
from pyspark.sql.types import *

import argparse
from multiprocessing import Pool
import sys, os, json, shutil, glob, warnings, gc, random, collections, csv, re, logging, time, pickle
from pathlib import Path
from datetime import datetime
from google.cloud import storage

import pandas as pd
import numpy as np
import lightgbm as lgb
import gcsfs

from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from utils.utils import *

/opt/conda/anaconda/lib/python3.6/site-packages/distributed/config.py:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config.update(yaml.load(text) or {})


In [4]:
def wmae(y, y_pred, weights):
    error = np.abs(y - y_pred)*weights
    return np.sum(error)/np.sum(weights)
def wrmse(y, y_pred, weights):
    error = np.power(y - y_pred, 2)*weights
    return np.sqrt(np.sum(error)/np.sum(weights))
def train(df_trn, df_val, feature_cols, target_col, lgb_params, model_name, weight_col=None):
    # build dataset
    if weight_col:
        dtrain = lgb.Dataset(df_trn[feature_cols],
                             df_trn[target_col].values,
                             weight=df_trn[weight_col].values,
                             #weight=np.log(df_trn[weight_col].values + np.e),
                             free_raw_data=False,
                             silent=False,
                             )
        dvalid = lgb.Dataset(df_val[feature_cols],
                             df_val[target_col].values,
                             weight=df_val[weight_col].values,
                             #weight=np.log(df_val[weight_col].values + np.e),
                             free_raw_data=False,
                             silent=False,
                             )
    # train model
    model = lgb.train(lgb_params,
                      dtrain,
                      valid_sets=[dtrain, dvalid],
                      verbose_eval=100)

    local_file = f'dp_output/{model_name}.txt'
    model.save_model(local_file, num_iteration=model.best_iteration)


def eval(df, feature_cols, target_col, model_name, weight_col=None):
    try:
        model = lgb.Booster(model_file=(f'dp_output/{model_name}.txt'))
    except:
        raise RuntimeError(f'{model_name} does not exist...')
    y_pred = model.predict(df[feature_cols])
    y_pred = np.array([max(0,x) if np.isnan(x)==False else x for x in y_pred])
    WRMSE = wrmse(df[target_col].values, y_pred, df[weight_col].values)
    WMAE = wmae(df[target_col].values, y_pred, df[weight_col].values)
    return y_pred

lgb_params = {
'n_estimators': 4000,
'metric': 'rmse',
'learning_rate': 0.02,
'objective': 'regression',
'boosting_type': 'gbdt',
'subsample': 0.9,
'bagging_freq': 10,
'colsample_bytree': 0.33,
'verbose': -1,
'num_leaves': 63,
'max_depth': 10,
'seed': 42,
'n_jobs': -1,
'early_stopping_rounds': 200,
}
config = load_json('gs://msc_fair_airflow/rpc_model/config_rpc.json')
adid_name = config["hash_id"]["adid"]
is_mobile_name = config["hash_id"]["is_mobile"]
source_id_name = config["hash_id"]["source_id"]

In [19]:
df = pd.read_csv('gs://msc_fair_airflow/rpc_model/sample_data_individual_feat/catalog_w1_2023-01-16.csv', usecols = ['uber_division_w1','uber_super_dept_w1',
                                                                                                                         'uber_dept_w1','uber_cat_w1',
                                                                                                                         'uber_subcat_w1','seller_id'])

In [14]:
df1 = pd.read_csv('gs://msc_fair_airflow/rpc_model/sample_data_individual_feat/catalog_w1_2023-01-16.csv',nrows = 1)

In [15]:
df1.columns

Index(['Unnamed: 0', 'catalog_item_id', 'seller_id', 'uber_division_w1',
       'uber_super_dept_w1', 'uber_dept_w1', 'uber_cat_w1', 'uber_subcat_w1',
       'uber_curr_item_price_w1', 'uber_num_appr_reviews_w1',
       'uber_avg_overall_rating_w1', 'uber_price_change_ratio_w1',
       'uber_price_change_value_w1', 'uber_avg_margin_w1'],
      dtype='object')

In [20]:
for i in ['uber_division_w1','uber_super_dept_w1',
                                                                                                                         'uber_dept_w1','uber_cat_w1',
                                                                                                                         'uber_subcat_w1','seller_id']:
    print(i,df[i].nunique())

uber_division_w1 10
uber_super_dept_w1 60
uber_dept_w1 529
uber_cat_w1 1635
uber_subcat_w1 5760
seller_id 39813


In [24]:
len(df[df['uber_subcat_w1'].isnull()==True])

796721

In [21]:
len(df)

3128108

In [41]:
today_ds = '2023-01-16'
partitionDate = spark.sql("""show partitions dmas.dfm""").rdd.flatMap(lambda x:x).map(lambda x : x.replace("ts=","")).max()



query = spark.sql("""SELECT dctr.*,
                            coalesce(pcoc.catalog_item_id, catalog_item_id_dctr) as catalog_item_id,
                            coalesce(pcoc.us_seller_id, seller_id_dctr) as seller_id,
                            dmas.ad_item_brand brand_nm,
                            darm.division_id AS division_id,
                            darm.super_department_id AS super_department_id,
                            darm.department_id AS department_id,
                            darm.category_id AS category_id,
                            darm.sub_category_id AS sub_category_id

                    FROM (SELECT DISTINCT adid, source_id, is_mobile, seller_id as seller_id_dctr, split(adid,'_')[0] as catalog_item_id_dctr
                          FROM sem.daily_criteria_traffic_revenue_v2_tmp
                          WHERE 
                               customer_id_p = '1' 
                               AND (source_id = 2 or source_id=4)
                               AND adtype_p ='pla' 
                               AND structure_version_p = 2
                               AND revenue_source_p = 'fair15'
                               AND date_string_p >= date_sub('__today__', 139) 
                               AND date_string_p <= date_sub('__today__', 7)) dctr

                          LEFT JOIN sem.pla_catalog_offer_criteria_shopping pcoc
                          ON dctr.adid = pcoc.adid


                          LEFT JOIN(
                                    SELECT adid, last(division_id) as division_id,
                                    last(sub_category_id) as sub_category_id, last(super_department_id) as super_department_id,
                                    last(department_id) as department_id, last(category_id) as category_id
                                    FROM sem.daily_adid_rh_metrics_v2 
                                    WHERE customer_id_p=1
                                    AND structure_version_p = 2
                                    AND revenue_source_p = 'fair15'
                                    AND date_string >= date_sub('__today__', 139) 
                                    AND date_string <= date_sub('__today__', 7)
                                    GROUP BY adid
                                    ) darm
                        ON dctr.adid = darm.adid 
                        
                          LEFT JOIN (SELECT item_id as catlg_item_id,
                                    max(lower(REGEXP_REPLACE(item_brand_name, '[^0-9A-Za-z]', ''))) as ad_item_brand
                                    FROM dmas.dfm
                                    WHERE ts=__partitionDate__
                                    GROUP BY item_id) dmas
                          ON coalesce(pcoc.catalog_item_id, dctr.catalog_item_id_dctr) = dmas.catlg_item_id
                          """.replace('__today__', today_ds).replace('__partitionDate__', partitionDate))

query.createOrReplaceTempView("dctr_pla")

In [42]:

days_backtrack_end = 6
days_backtrack_start = 0
bad_days = """('2022-11-07', '2022-11-14', '2022-11-21', '2022-11-23', '2022-11-24', '2022-11-25')"""
query = """WITH             
    dctr
    AS
    (SELECT dctr_v2.adid,
            dctr_v2.source_id,
            dctr_v2.is_mobile,
            SUM(impressions) AS impressions,
            SUM(clicks) AS clicks,
            SUM(adspend) AS adspend,
            SUM(qty) AS qty,
            MAX(att_lineitems_brand) AS orders_brand,
            MAX(att_lineitems_seller) AS orders_seller,
            MAX(attributed_gmv_brand) AS gmv_revenue_adjustment_brand,
            MAX(attributed_gmv_seller) AS gmv_revenue_adjustment_seller,
            -- SUM(orders) AS orders,
            -- SUM(gmv_revenue_adjustment) AS gmv_revenue_adjustment,
            SUM(cost) AS cost,
            SUM(estimated_cp) AS estimated_cp

    FROM (SELECT * FROM sem.daily_criteria_traffic_revenue_v2_tmp
           WHERE customer_id_p = '1' 
           AND (source_id = 2 or source_id=4)
           AND adtype_p ='pla' 
           AND structure_version_p = 2
           AND revenue_source_p = 'fair15'
           AND date_string_p >= date_sub('__today__', __daysback_end__) 
           AND date_string_p <= date_sub('__today__', __daysback_start__) 
           AND date_string_p NOT IN __bad_days__) dctr_v2
    LEFT JOIN (
    SELECT adid, is_mobile, source_id, SUM(att_lineitems_brand) as att_lineitems_brand, SUM(att_lineitems_seller) as att_lineitems_seller,
            SUM(attributed_gmv_brand) as attributed_gmv_brand, SUM(attributed_gmv_seller) as attributed_gmv_seller
    FROM fair_dev.pt3_14day_report_nas_test 
    WHERE
            ad_date >= date_sub('__today__', __daysback_end__) 
            AND ad_date <= date_sub('__today__', __daysback_start__) 
            AND ad_date NOT IN __bad_days__ 
            AND adid != 'NULL'
            AND is_pla = 1
    GROUP BY adid, is_mobile, source_id
    ) coop_14d
    ON dctr_v2.adid = coop_14d.adid
    AND dctr_v2.is_mobile = coop_14d.is_mobile
    AND dctr_v2.source_id = coop_14d.source_id

    GROUP BY dctr_v2.adid, dctr_v2.is_mobile, dctr_v2.source_id)

       --final query
       SELECT  dctr_pla.catalog_item_id AS catalog_item_id,
               dctr_pla.seller_id AS seller_id,
               dctr_pla.source_id AS source_id,
               dctr_pla.adid AS adid,
               dctr_pla.brand_nm AS brand_nm,
               dctr_pla.division_id AS division_id,
               dctr_pla.super_department_id AS super_department_id,
               dctr_pla.department_id AS department_id,
               dctr_pla.category_id AS category_id,
               dctr_pla.sub_category_id AS sub_category_id,
               dctr_pla.is_mobile AS is_mobile,
               dctr.impressions/__tot_days__ AS impressions,
               dctr.clicks/__tot_days__ AS clicks,
               dctr.adspend/__tot_days__ AS adspend,
               dctr.qty/__tot_days__ AS qty,
               dctr.orders_brand/__tot_days__ AS orders_brand,
               dctr.orders_seller/__tot_days__ AS orders_seller,
               dctr.gmv_revenue_adjustment_seller/__tot_days__ AS gmv_revenue_adjustment_seller,
               dctr.gmv_revenue_adjustment_brand/__tot_days__ AS gmv_revenue_adjustment_brand,
               dctr.cost/__tot_days__ AS cost,
               (dctr.gmv_revenue_adjustment_seller - dctr.cost)/__tot_days__ AS margin_seller,
               (dctr.gmv_revenue_adjustment_brand - dctr.cost)/__tot_days__ AS margin_brand,
               dctr.estimated_cp/__tot_days__ AS estimated_cp,
               dctr.clicks/dctr.impressions AS ctr,
               dctr.adspend/dctr.clicks AS cpc,
               dctr.gmv_revenue_adjustment_brand/dctr.clicks AS rpc_brand,
               dctr.gmv_revenue_adjustment_seller/dctr.clicks AS rpc_seller,
               dctr.gmv_revenue_adjustment_brand/dctr.adspend AS roas_brand,
               dctr.gmv_revenue_adjustment_seller/dctr.adspend AS roas_seller,
               dctr.orders_brand/dctr.clicks AS convrt_brand,
               dctr.orders_seller/dctr.clicks AS convrt_seller,
               dctr.gmv_revenue_adjustment_brand/dctr.orders_brand AS ordersize_brand,
               dctr.gmv_revenue_adjustment_seller/dctr.orders_seller AS ordersize_seller,
               dctr.estimated_cp/dctr.orders_brand AS cpsize_brand,
               dctr.estimated_cp/dctr.orders_seller AS cpsize_seller,
               (dctr.gmv_revenue_adjustment_brand - dctr.cost)/dctr.clicks AS mpc_brand,
               (dctr.gmv_revenue_adjustment_seller - dctr.cost)/dctr.clicks AS mpc_seller,
               (dctr.gmv_revenue_adjustment_brand - dctr.cost)/dctr.orders_brand AS mpo_brand,
               (dctr.gmv_revenue_adjustment_seller - dctr.cost)/dctr.orders_seller AS mpo_seller,
               dctr.gmv_revenue_adjustment_brand/dctr.estimated_cp AS rpcp_brand,
               dctr.gmv_revenue_adjustment_seller/dctr.estimated_cp AS rpcp_seller

        FROM dctr_pla LEFT JOIN dctr
             ON dctr_pla.adid = dctr.adid
             AND dctr_pla.source_id = dctr.source_id
             AND dctr_pla.is_mobile = dctr.is_mobile
        """

query = query.replace('__today__', today_ds)
query = query.replace('__daysback_end__', str(days_backtrack_end))
query = query.replace('__daysback_start__', str(days_backtrack_start))
query = query.replace('__bad_days__', bad_days)
bad_days_list = [i.replace("('",'').replace("')",'') for i in bad_days.split("', '")]
exclude_days = len([datetime.strptime(dat, '%Y-%m-%d') for dat in bad_days_list if datetime.strptime(dat, '%Y-%m-%d').date()>=datetime.strptime(today_ds, '%Y-%m-%d').date()- \
             timedelta(days=days_backtrack_end) and datetime.strptime(dat, '%Y-%m-%d').date()<=datetime.strptime(today_ds, '%Y-%m-%d').date()- timedelta(days=days_backtrack_start)])
query = query.replace('__tot_days__', str(days_backtrack_end - days_backtrack_start+1-exclude_days))

kk = spark.sql(query).toPandas()

In [43]:
len(kk)

5186948

In [52]:
len(kk[(kk['rpc_brand']!=0)&(kk['rpc_brand'].isnull()==False)])

13484

In [16]:
spark.sql('show partitions casesci_sem.items_w_dups_mapping_sem_offer_bidding').collect()

[Row(partition='date_string=2022-03-14'),
 Row(partition='date_string=2022-11-04'),
 Row(partition='date_string=latest')]

In [17]:
spark.sql("select * from casesci_sem.items_w_dups_mapping_sem_offer_bidding where date_string='latest'").show()

+----------+--------------------+--------------+-----------------+------------+-----------+
|   item_id|        product_type|id_unique_item|id_unique_variant|  product_id|date_string|
+----------+--------------------+--------------+-----------------+------------+-----------+
| 206072312|bicycle_bottom_br...|    PT971U0878|   PT971U0878V000|0RCN4SGTQ8D8|     latest|
| 301700425|art_stencils_and_...|  PT4095U14794| PT4095U14794V000|0RCNEX2NP734|     latest|
| 939541820|  art_pads_and_paper|   PT3743U3259|  PT3743U3259V000|0RCNM2Q1J8BV|     latest|
|1380922101| christmas_stockings|    PT973U2885|   PT973U2885V001|0RCO6DIEZQAL|     latest|
| 632401383|    cell_phone_cases|  PT012U652213| PT012U652213V001|0RCO7I3GIA8C|     latest|
|1280002434|            t_shirts|  PT001U236191| PT001U236191V003|0RCOGTIN8MNB|     latest|
|1792852021|   disposable_gloves|   PT3509U0424|  PT3509U0424V001|0RCOIFC7ZNTG|     latest|
|1466735574|    blouses_and_tops|  PT000U360019| PT000U360019V007|0RCOW2R1MG1M| 

In [18]:
spark.sql("select * from casesci_sem.items_w_dups_mapping_sem_offer_bidding where date_string='latest'").count()

454891131

In [23]:
spark.sql("select distinct * from casesci_sem.items_w_dups_mapping_sem_offer_bidding where date_string='latest'").count()

454891131

In [20]:
spark.sql("select count(distinct item_id) from casesci_sem.items_w_dups_mapping_sem_offer_bidding where date_string='latest'").show()

+-----------------------+
|count(DISTINCT item_id)|
+-----------------------+
|              454823627|
+-----------------------+



In [21]:
spark.sql("select count(distinct id_unique_item) from casesci_sem.items_w_dups_mapping_sem_offer_bidding where date_string='latest'").show()

+------------------------------+
|count(DISTINCT id_unique_item)|
+------------------------------+
|                      61508686|
+------------------------------+



In [22]:
spark.sql("select count(distinct id_unique_variant) from casesci_sem.items_w_dups_mapping_sem_offer_bidding where date_string='latest'").show()

+---------------------------------+
|count(DISTINCT id_unique_variant)|
+---------------------------------+
|                        138688922|
+---------------------------------+



In [ ]:
    item_agg.write.mode("overwrite").insertInto(output_hive_table, overwrite=True)
    spark.catalog.clearCache()
    spark._jsparkSession.sharedState().cacheManager().clearCache()

In [59]:
[i for i in df.columns if 'site' in i]

['site_total_page_views_w1',
 'site_total_add_to_carts_w1',
 'site_total_orders_w1',
 'site_total_revenue_w1',
 'site_atcpv_w1',
 'site_convrt_w1',
 'site_rpv_w1',
 'site_mpv_w1',
 'site_total_page_views_w2',
 'site_total_add_to_carts_w2',
 'site_total_orders_w2',
 'site_total_revenue_w2',
 'site_atcpv_w2',
 'site_convrt_w2',
 'site_rpv_w2',
 'site_mpv_w2']

In [6]:
# for i in ['uber_super_dept_w1','uber_division_w1','uber_dept_w1','uber_cat_w1','uber_subcat_w1']:
#     df[i] = df[i].astype(str)
for i in ['uber_curr_item_price_w1','uber_avg_overall_rating_w1']:
    df[i] = df[i].apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [5]:
sdf = spark.read.format("csv").option("header","true").load('gs://msc_fair_airflow/rpc_model/sample_data_full_feat/df_feat_all_coop_item_2023-01-16.csv')

In [7]:
from pyspark.sql.types import IntegerType,BooleanType,DateType, FloatType
sdf=sdf.withColumn("uber_curr_item_price_w1",sdf.uber_curr_item_price_w1.cast(FloatType()))
sdf=sdf.withColumn("uber_avg_overall_rating_w1",sdf.uber_avg_overall_rating_w1.cast(FloatType()))

In [8]:
partitionDate = spark.sql("""show partitions dmas.dfm""").rdd.flatMap(lambda x:x).map(lambda x : x.replace("ts=","")).max()

item_info_dfm = spark.sql("""SELECT item_id as catlg_item_id,
                                        max(lower(REGEXP_REPLACE(item_brand_name, '[^0-9A-Za-z]', ''))) as ad_item_brand
                FROM dmas.dfm
                WHERE ts='"""+partitionDate+"""'
                GROUP BY item_id""")

sdf = sdf.join(item_info_dfm,sdf.catalog_item_id == item_info_dfm.catlg_item_id,how='left').drop(item_info_dfm.catlg_item_id)

In [9]:
sdf = sdf.withColumn('weight_price_cart_w1', sdf.site_total_add_to_carts_w1*(sdf.uber_curr_item_price_w1-sdf.uber_price_change_value_w1))
sdf = sdf.withColumn('weight_price_cart_w2', sdf.site_total_add_to_carts_w2*(sdf.uber_curr_item_price_w1-sdf.uber_price_change_value_w1))

sdf = sdf.withColumn('weight_price_orders_w1', sdf.site_total_orders_w1*(sdf.uber_curr_item_price_w1-sdf.uber_price_change_value_w1))
sdf = sdf.withColumn('weight_price_orders_w2', sdf.site_total_orders_w2*(sdf.uber_curr_item_price_w1-sdf.uber_price_change_value_w1))

In [10]:
sdf.createOrReplaceTempView('sdf')

In [13]:
spark.sql("""SELECT sdf.*, brand_group.brand_weight_price_cart_w1,brand_group.brand_weight_price_cart_w2,
                            brand_group.brand_weight_price_orders_w1,brand_group.brand_weight_price_orders_w2,
                            seller_group.seller_weight_price_cart_w1,seller_group.seller_weight_price_cart_w2,
                            seller_group.seller_weight_price_orders_w1,seller_group.seller_weight_price_orders_w2
            FROM sdf 
            LEFT JOIN (select ad_item_brand, sum(weight_price_cart_w1)/sum(site_total_add_to_carts_w1) as brand_weight_price_cart_w1,
              sum(weight_price_cart_w2)/sum(site_total_add_to_carts_w2) as brand_weight_price_cart_w2,
              sum(weight_price_orders_w1)/sum(site_total_orders_w1) as brand_weight_price_orders_w1,
              sum(weight_price_orders_w2)/sum(site_total_orders_w2) as brand_weight_price_orders_w2
              from sdf
              group by ad_item_brand) brand_group
            ON sdf.ad_item_brand = brand_group.ad_item_brand
            LEFT JOIN (select seller_id, sum(weight_price_cart_w1)/sum(site_total_add_to_carts_w1) as seller_weight_price_cart_w1,
              sum(weight_price_cart_w2)/sum(site_total_add_to_carts_w2) as seller_weight_price_cart_w2,
              sum(weight_price_orders_w1)/sum(site_total_orders_w1) as seller_weight_price_orders_w1,
              sum(weight_price_orders_w2)/sum(site_total_orders_w2) as seller_weight_price_orders_w2
              from sdf
              group by seller_id) seller_group
            ON sdf.seller_id = seller_group.seller_id""").coalesce(1).write.option("header", "true").csv('gs://msc_fair_airflow/rpc_model/sample_data_full_feat/df_feat_all_coop_item_2023-01-16_new.csv')



In [5]:
df = pd.read_csv('gs://msc_fair_airflow/rpc_model/sample_data_full_feat/df_feat_all_coop_item_2023-01-16_new.csv/part-00000-b2887bfb-6832-4641-a1dd-732184f65274-c000.csv')

/opt/conda/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (245,246,247,248,249,284) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df = df.sort_values('sem_clicks_w1',ascending=False)
df = df.head(500000)

In [7]:
for i in [k for k in df.columns if '_l' in k]:
    df[i] = df[i].fillna(0)
df_eval = df.copy()
# prepare train features
features = config["features"]
# clean features (all should be numerics)
features_set = ['uber_curr_item_price_w1', 'uber_avg_overall_rating_w1'] #hard coded for hot fix
df = convert_numeric(df, features_set)

# do train-val split
df['fold'] = 0
kf = KFold(5, shuffle=True, random_state=42)
for i, (_, val_idx) in enumerate(kf.split(df)):
    df['fold'].iloc[val_idx] = i
    
# filters
convrt_trn_brand_filter = (df[source_id_name]==2) & (df.sem_convrt_brand_l>=0) & (df.sem_convrt_brand_l<=0.3)
convrt_val_brand_filter = (df[source_id_name]==2) & (df.sem_convrt_brand_l>=0) & (df.sem_convrt_brand_l<=0.3)
#    ordersize_trn_filter = (df[source_id_name]==2) & (df.sem_orders_l>=3) & (df.sem_ordersize_l>=0) & (df.sem_ordersize_l<=500)
ordersize_trn_brand_filter = (df[source_id_name]==2) & (df.sem_ordersize_brand_l>=0) & (df.sem_ordersize_brand_l<=500)
ordersize_val_brand_filter = (df[source_id_name]==2) & (df.sem_ordersize_brand_l>=0) & (df.sem_ordersize_brand_l<=500)
rpc_trn_brand_filter = (df[source_id_name]==2) & (df.sem_ordersize_brand_l>=0) & (df.sem_ordersize_brand_l<=500)
rpc_val_brand_filter = (df[source_id_name]==2) & (df.sem_ordersize_brand_l>=0) & (df.sem_ordersize_brand_l<=500)

convrt_trn_seller_filter = (df[source_id_name]==2) & (df.sem_convrt_seller_l>=0) & (df.sem_convrt_seller_l<=0.3)
convrt_val_seller_filter = (df[source_id_name]==2) & (df.sem_convrt_seller_l>=0) & (df.sem_convrt_seller_l<=0.3)
#    ordersize_trn_filter = (df[source_id_name]==2) & (df.sem_orders_l>=3) & (df.sem_ordersize_l>=0) & (df.sem_ordersize_l<=500)
ordersize_trn_seller_filter = (df[source_id_name]==2) & (df.sem_ordersize_seller_l>=0) & (df.sem_ordersize_seller_l<=500)
ordersize_val_seller_filter = (df[source_id_name]==2) & (df.sem_ordersize_seller_l>=0) & (df.sem_ordersize_seller_l<=500)
rpc_trn_seller_filter = (df[source_id_name]==2) & (df.sem_ordersize_seller_l>=0) & (df.sem_ordersize_seller_l<=500)
rpc_val_seller_filter = (df[source_id_name]==2) & (df.sem_ordersize_seller_l>=0) & (df.sem_ordersize_seller_l<=500)
filter_dic = {'convrt':{'brand':[convrt_trn_brand_filter, convrt_val_brand_filter], 'seller':[convrt_trn_seller_filter,convrt_val_seller_filter]},\
              'ordersize':{'brand':[ordersize_trn_brand_filter, ordersize_val_brand_filter], 'seller':[ordersize_trn_seller_filter,ordersize_val_seller_filter]},\
              'rpc':{'brand':[rpc_trn_brand_filter, rpc_val_brand_filter], 'seller':[rpc_trn_seller_filter,rpc_val_seller_filter]}}

df_eval['pred_convrt_brand'] = np.nan
df_eval['pred_ordersize_brand'] = np.nan
df_eval['pred_rpc_brand'] = np.nan
df_eval['pred_convrt_seller'] = np.nan
df_eval['pred_ordersize_seller'] = np.nan
df_eval['pred_rpc_seller'] = np.nan

targets = ['seller', 'brand']

/opt/conda/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
tar='seller'
[tar+'_'+i for i in ['weight_price_cart_w1','weight_price_cart_w2','weight_price_orders_w1','weight_price_orders_w2']]

['seller_weight_price_cart_w1',
 'seller_weight_price_cart_w2',
 'seller_weight_price_orders_w1',
 'seller_weight_price_orders_w2']

In [10]:
for tar in targets:
    for fold in range(5):
        print('model'+str(fold))
        # convrt model
        model_features = features['convrt'][tar]+[tar+'_'+i for i in ['weight_price_cart_w1','weight_price_cart_w2','weight_price_orders_w1','weight_price_orders_w2']]
        target_col = "sem_convrt_"+tar+"_l"
        model_name = f'convrt_f{fold}'+'_'+tar
        trn_mask = filter_dic['convrt'][tar][0] & (df.fold != fold)
        df_trn = df[trn_mask]
        val_mask = filter_dic['convrt'][tar][1] & (df.fold == fold)
        df_val = df[val_mask]
        train(df_trn, df_val, model_features, target_col, lgb_params, model_name, weight_col='sem_clicks_l')

        # Evaluate
        y_pred = eval(df_val, model_features, target_col, model_name, weight_col='sem_clicks_l')
        df_eval['pred_convrt_'+tar].loc[val_mask] = y_pred

        # order_size model
        model_features = features['ordersize'][tar]+[tar+'_'+i for i in ['weight_price_cart_w1','weight_price_cart_w2','weight_price_orders_w1','weight_price_orders_w2']]
        target_col = "sem_ordersize_"+tar+"_l"
        model_name = f'ordersize_f{fold}'+'_'+tar
        trn_mask = filter_dic['ordersize'][tar][0] & (df.fold != fold)
        df_trn = df[trn_mask]
        val_mask = filter_dic['ordersize'][tar][1] & (df.fold == fold)
        df_val = df[val_mask]
        train(df_trn, df_val, model_features, target_col, lgb_params, model_name, weight_col='sem_orders_'+tar+'_l')

        # Evaluate
        y_pred = eval(df_val, model_features, target_col, model_name, weight_col='sem_orders_'+tar+'_l')
        df_eval['pred_ordersize_'+tar].loc[val_mask] = y_pred

model0


/home/y0c07y1/.local/lib/python3.6/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/y0c07y1/.local/lib/python3.6/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/home/y0c07y1/.local/lib/python3.6/site-packages/lightgbm/basic.py:1491: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via 'params' instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 200 rounds
[100]	training's rmse: 0.01552	valid_1's rmse: 0.0267574
[200]	training's rmse: 0.0115921	valid_1's rmse: 0.0266837
Early stopping, best iteration is:
[63]	training's rmse: 0.0188295	valid_1's rmse: 0.0265497
Training until validation scores don't improve for 200 rounds
[100]	training's rmse: 10.72	valid_1's rmse: 31.0617
[200]	training's rmse: 6.17085	valid_1's rmse: 32.3838
Early stopping, best iteration is:
[60]	training's rmse: 14.382	valid_1's rmse: 29.4983
model1
Training until validation scores don't improve for 200 rounds
[100]	training's rmse: 0.0151784	valid_1's rmse: 0.0315396
[200]	training's rmse: 0.0114777	valid_1's rmse: 0.0313758
[300]	training's rmse: 0.00979235	valid_1's rmse: 0.0312835
[400]	training's rmse: 0.00834402	valid_1's rmse: 0.0313015
[500]	training's rmse: 0.00718451	valid_1's rmse: 0.0313026
Early stopping, best iteration is:
[376]	training's rmse: 0.00866863	valid_1's rmse: 0.0312822
Training 

In [11]:
for tar in targets:
    df_eval['pred_rpc_'+tar] = df_eval['pred_convrt_'+tar] * df_eval['pred_ordersize_'+tar]
    target_cols = ['sem_convrt_'+tar+'_l', 'sem_ordersize_'+tar+'_l', 'sem_rpc_'+tar+'_l']
    pred_cols = ['pred_convrt_'+tar, 'pred_ordersize_'+tar, 'pred_rpc_'+tar]
    weight_cols = ['sem_clicks_l', 'sem_orders_'+tar+'_l', 'sem_clicks_l']
    for target_col, pred_col, weight_col in zip(target_cols, pred_cols, weight_cols):
        filter_NoNA = df_eval[target_col].notnull() & df_eval[pred_col].notnull() & df_eval[weight_col].notnull()
        WRMSE = wrmse(df_eval[target_col].loc[filter_NoNA].values, df_eval[pred_col].loc[filter_NoNA].values, df_eval[weight_col].loc[filter_NoNA].values)
        WMAE = wmae(df_eval[target_col].loc[filter_NoNA].values, df_eval[pred_col].loc[filter_NoNA].values, df_eval[weight_col].loc[filter_NoNA].values)
        print(f"{'_'.join(pred_col.split('_')[1:-1])} oof based on lightgbm WRMSE: {WRMSE: .5f}")
        print(f"{'_'.join(pred_col.split('_')[1:-1])} oof based on lightgbm WMAE: {WMAE: .5f}")

convrt oof based on lightgbm WRMSE:  0.02942
convrt oof based on lightgbm WMAE:  0.01457
ordersize oof based on lightgbm WRMSE:  28.13425
ordersize oof based on lightgbm WMAE:  14.04540
rpc oof based on lightgbm WRMSE:  0.94024
rpc oof based on lightgbm WMAE:  0.29411
convrt oof based on lightgbm WRMSE:  0.01702
convrt oof based on lightgbm WMAE:  0.00640
ordersize oof based on lightgbm WRMSE:  43.64110
ordersize oof based on lightgbm WMAE:  23.82708
rpc oof based on lightgbm WRMSE:  1.68347
rpc oof based on lightgbm WMAE:  0.49071


# deep learning

In [3]:
from deepctr_torch.models import DIEN
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [4]:
import pandas as pd
import torch
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from deepctr_torch.inputs import SparseFeat, get_feature_names
from deepctr_torch.models import DeepFM

In [9]:
data = pd.read_csv('gs://msc_fair_airflow/rpc_model/sample_data_full_feat/df_feat_all_coop_item_2023-01-16.csv',nrows = 10000)
sparse_features = ['uber_dept_w1',
 'uber_subcat_w1',
 'uber_super_dept_w1',
 'uber_division_w1',
 'uber_cat_w1']
target = ['sem_rpc_seller_l']

In [17]:
config = load_json('gs://msc_fair_airflow/rpc_model/config_rpc.json')
adid_name = config["hash_id"]["adid"]
is_mobile_name = config["hash_id"]["is_mobile"]
source_id_name = config["hash_id"]["source_id"]
features = config['features']
model_features = features['convrt']['seller']

In [ ]:
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
# 2.count #unique features for each sparse field
fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique())
                          for feat in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [24]:
[i for i in list(set(data.columns)-set(model_features)) if '_brand_' not in i and '_l' not in i]

['uber_dept_w1',
 'uber_subcat_w1',
 'catalog_item_id',
 'source_id',
 'seller_id',
 'adid',
 'uber_super_dept_w1',
 'uber_division_w1',
 'uber_cat_w1']

In [5]:
data = pd.read_csv("./movielens_sample.txt")
sparse_features = ["movie_id", "user_id",
                   "gender", "age", "occupation", "zip"]
target = ['rating']

# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
# 2.count #unique features for each sparse field
fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique())
                          for feat in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 3.generate input data for model
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}
# 4.Define Model,train,predict and evaluate

device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

In [7]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression', device=device)
model.compile("adam", "mae", metrics=['htdhtr'], )

history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=10, verbose=2,
                    validation_split=0.2)
pred_ans = model.predict(test_model_input, batch_size=256)

cpu
Train on 128 samples, validate on 32 samples, 1 steps per epoch
Epoch 1/10
0s - loss:  3.5633
Epoch 2/10
0s - loss:  3.5127
Epoch 3/10
0s - loss:  3.4692
Epoch 4/10
0s - loss:  3.4275
Epoch 5/10
0s - loss:  3.3875
Epoch 6/10
0s - loss:  3.3485
Epoch 7/10
0s - loss:  3.3075
Epoch 8/10
0s - loss:  3.2645
Epoch 9/10
0s - loss:  3.2192
Epoch 10/10
0s - loss:  3.1717


In [26]:
train

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
92,76,49,3,974658834,"20,000 Leagues Under the Sea (1954)",Adventure|Children's|Fantasy|Sci-Fi,1,5,15,15
184,71,3,5,974697744,Dead Man Walking (1995),Drama,1,6,12,163
11,7,173,5,976316283,High Fidelity (2000),Comedy,1,2,2,140
27,73,100,4,974677761,On the Waterfront (1954),Crime|Drama,0,6,12,9
30,65,122,2,974699637,"Player, The (1992)",Comedy|Drama,1,1,0,162
...,...,...,...,...,...,...,...,...,...,...
22,175,137,3,960328279,"Rocky Horror Picture Show, The (1975)",Comedy|Horror|Musical|Sci-Fi,1,4,7,3
187,67,186,5,974753321,Meet the Parents (2000),Comedy,1,2,7,151
42,143,101,4,965094644,"Last Emperor, The (1987)",Drama|War,1,5,0,1
141,171,80,4,961055887,Star Trek: The Wrath of Khan (1982),Action|Adventure|Sci-Fi,1,2,11,161


In [21]:
pd.read_csv("./movielens_sample.txt")['user_id'].nunique()

193